In [83]:
from database import SessionLocal
from models import Category, Post
from sqlalchemy import func

import random

In [ ]:
def create_category(name, parent_id=None):
    session = SessionLocal()
    category = Category(name=name, parent_id=parent_id)
    session.add(category)
    session.commit()
    session.close()

def delete_category_by_id(category_id):
    session = SessionLocal()
    category = session.query(Category).filter_by(id=category_id).first()
    
    if not category:
        print(f"Category with ID={category_id} not found.")
        session.close()
        return None
    
    # Delete all posts in this category
    posts = session.query(Post).filter_by(category_id=category_id)
    if posts.exists():
        print(f"Would delete {posts.count()} posts in category ID={category_id}, aborting.")
        session.close()
        return None
    posts.delete()
    
    # Delete the category itself
    session.delete(category)
    session.commit()
    session.close()
    
    print(f"Category ID={category_id} deleted successfully.")
    return category

def edit_category_by_id(category_id, new_name=None, new_parent_id=False):
    """
    Edit an existing category's name or parent category.
    
    :param category_id: (int) The ID of the category to edit.
    :param new_name: (str) New name for the category (optional).
    :param new_parent_id: (int or None) New parent category ID (optional).
                          Use None to make it a root category.
    :return: The updated Category object, or None if not found.
    """
    session = SessionLocal()
    category = session.query(Category).filter_by(id=category_id).first()
    
    if not category:
        print(f"Category with ID={category_id} not found.")
        session.close()
        return None
    
    # Update fields if provided
    if new_name is not None:
        category.name = new_name
    
    if new_parent_id is not False:
        # Validate new_parent_id is not the same as category_id
        if new_parent_id == category_id:
            print("Error: Cannot assign the category as its own parent.")
            session.close()
            return None
        
        # Optional: You could also check for cyclical references.
        # For a simple example, we’ll skip that logic here.
        category.parent_id = new_parent_id
    
    session.commit()
    updated = category  # hold reference
    session.close()
    
    print(f"Category ID={category_id} updated successfully.")
    return updated

def edit_post_category_by_id(post_id, new_category_id):
    session = SessionLocal()
    post = session.query(Post).filter_by(id=post_id).first()
    
    if not post:
        print(f"Post with ID={post_id} not found.")
        session.close()
        return None
    
    post.category_id = new_category_id
    session.commit()
    updated = post  # hold reference
    session.close()
    
    print(f"Post ID={post_id} updated successfully.")
    return updated

create_category('space', 9)
# edit_category_by_id(24, new_name='entertainment')
# edit_category_by_id(10, new_parent_id=8)
# delete_category_by_id(6)

edit_post_category_by_id(953, 50)


In [ ]:
def get_random_uncategorized_post():
    session = SessionLocal()
    uncategorized_posts = session.query(Post).filter(Post.category_id == None).all()
    if not uncategorized_posts:
        return None
    return random.choice(uncategorized_posts)

def assign_category_to_post(post_id, category_id):
    session = SessionLocal()
    post = session.query(Post).filter_by(id=post_id).first()
    if post:
        post.category_id = category_id
        session.commit()
    session.close()

In [ ]:
def get_category_stats():
    """
    Returns a list of dictionaries, each describing a category's
    total post count (including all nested subcategories).
    
    Each list item includes:
      - 'category_id'
      - 'name'
      - 'full_path': e.g. "economy -> investing -> bitcoin"
      - 'total_posts'
    """
    session = SessionLocal()
    
    # 1. Load all categories
    categories = session.query(Category).all()
    
    # Build maps for easy lookup
    cat_map = {c.id: c for c in categories}  # id -> Category object
    children_map = {}
    for c in categories:
        # group children by parent_id
        parent_id = c.parent_id
        if parent_id not in children_map:
            children_map[parent_id] = []
        children_map[parent_id].append(c.id)
    
    # 2. Compute direct post counts per category
    # Example: {1: 5, 2: 2, ...} means cat 1 has 5 posts, cat 2 has 2 posts, etc.
    post_counts = dict(
        session.query(Post.category_id, func.count(Post.id))
               .group_by(Post.category_id)
               .all()
    )
    
    session.close()
    
    # 3. Recursive function to sum up post counts for a category + its subcategories
    def get_total_posts(cat_id):
        direct_count = post_counts.get(cat_id, 0)
        total = direct_count
        for sub_id in children_map.get(cat_id, []):
            total += get_total_posts(sub_id)
        return total

    # 4. Gather stats in a hierarchical manner
    results = []
    
    # Find root categories (where parent_id is None)
    root_categories = [c for c in categories if c.parent_id is None]

    def gather_stats(cat_id, path=""):
        cat_obj = cat_map[cat_id]
        # Build a full_path like "economy -> investing -> bitcoin"
        full_path = path + " -> " + cat_obj.name if path else cat_obj.name
        
        total_posts = get_total_posts(cat_id)
        results.append({
            "category_id": cat_id,
            "name": cat_obj.name,
            "full_path": full_path,
            "total_posts": total_posts
        })
        
        # Recurse into subcategories
        for sub_id in children_map.get(cat_id, []):
            gather_stats(sub_id, full_path)

    for root_cat in root_categories:
        gather_stats(root_cat.id, "")

    return results

stats = get_category_stats()
for s in stats:
    print(
        f"ID={s['category_id']}, Path='{s['full_path']}', Posts={s['total_posts']}"
    )